In [1]:
from gensim.models import word2vec

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/raw/solr-dsi-domain_only-04-28-2017-cleaned.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524062 entries, 0 to 524061
Data columns (total 13 columns):
_version_    524062 non-null int64
cache        11882 non-null object
segment      524062 non-null int64
digest       524062 non-null object
tstamp       524062 non-null object
url          524062 non-null object
anchor       45801 non-null object
content      524062 non-null object
id           524062 non-null object
title        524062 non-null object
boost        524062 non-null float64
domain       524062 non-null object
title_c      518067 non-null object
dtypes: float64(1), int64(2), object(10)
memory usage: 52.0+ MB


In [5]:
df = df.dropna(subset =['content'])

In [6]:
dsi_stop_words = ['the', 'blog', 'i', 'in', 'new', 'use', 'a', 'how', 'it', 'like', 'need', 'sign', 'for', 
                  'rss', 'videos', 'view', 'using', 'interview', 'follow', 'read', 'make', 'video',
                  'post', 'comment', 'comments', 'subscribe', 'things', 'just', 'add', 'wise', 'know', 'upcoming', 
                  'people', 'practitioner', 'used', 'developers', 'events', 'companies', 'better', 'terms', 'time',
                  'customer', 'conference', 'tags', 'contact', 'years', 'type', 'value', 'march', 'best', 'does',
                 'live', 'cases', 'way', 'privacy']

In [9]:
from sklearn.feature_extraction import stop_words
import re
def dsi_title_letters_removal(x):
    #print len(x)
    #print x
    s = re.sub('[^a-zA-Z]+', '', x)
    #print s
    if(len(s) > 0):
        s = s.strip().lower()
        if s not in stop_words.ENGLISH_STOP_WORDS: # and s not in dsi_stop_words:
            return s
        else:
            return None
    return None

In [67]:
map_content2words(df.iloc[0]['content'])

['organization',
 'committee',
 'ecmlpkdd',
 'skip',
 'main',
 'content',
 'toggle',
 'navigation',
 'global',
 'info',
 'key',
 'dates',
 'organization',
 'committee',
 'sponsorship',
 'discovery',
 'challenges',
 'past',
 'conferences',
 'ecmlpkdd',
 'program',
 'preliminary',
 'schedule',
 'invited',
 'speakers',
 'workshop',
 'list',
 'tutorial',
 'list',
 'accepted',
 'papers',
 'submission',
 'publication',
 'model',
 'journal',
 'track',
 'scientific',
 'proceedings',
 'track',
 'industrial',
 'proceedings',
 'track',
 'tutorials',
 'workshops',
 'nectar',
 'track',
 'demos',
 'track',
 'doctoral',
 'consortium',
 'copyright',
 'form',
 'attending',
 'registration',
 'venue',
 'accommodation',
 'porto',
 'travel',
 'info',
 'summer',
 'school',
 'search',
 'home',
 'global',
 'info',
 'organization',
 'committee',
 'organization',
 'committee',
 'person',
 'chairs',
 'program',
 'chairs',
 'journal',
 'track',
 'chairs',
 'industrial',
 'track',
 'chairs',
 'local',
 'organizati

In [10]:
def map_content2words(x):
    s = x.split()
    #print s
    return [dsi_title_letters_removal(item) for item in s if dsi_title_letters_removal(item) != None]

In [11]:
from time import time
t0 = time()
df['title_w'] = df.title.map(lambda x: map_content2words(x))
time() - t0

13.029299020767212

In [12]:
from time import time
t0 = time()
df['content_w'] = df.content.map(lambda x: map_content2words(x))
time() - t0

1166.5438408851624

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 524062 entries, 0 to 524061
Data columns (total 13 columns):
_version_    524062 non-null int64
cache        11882 non-null object
segment      524062 non-null int64
digest       524062 non-null object
tstamp       524062 non-null object
url          524062 non-null object
anchor       45801 non-null object
content      524062 non-null object
id           524062 non-null object
title        524062 non-null object
boost        524062 non-null float64
domain       524062 non-null object
words        524062 non-null object
dtypes: float64(1), int64(2), object(10)
memory usage: 56.0+ MB


In [13]:
title_w = df['title_w']
word_w = df['content_w']
words = title_w + word_w

In [14]:
from time import time

In [15]:
t0 = time()
model = word2vec.Word2Vec(words, size=250, hs=1)
time() - t0

2575.600515127182

In [17]:
model.save("dsi_model_v5.bin")
#model = word2vec.Word2Vec.load('dsi_model_format_v2.bin')

In [18]:
model.wv.save_word2vec_format('dsi_model_format_v5.bin')

#from gensim.models import Word2Vec, KeyedVectors
#model = KeyedVectors.load_word2vec_format('../Words_Embedding/dsi_model_format_v1.bin', binary=True)

In [23]:
from gensim.models import Word2Vec, KeyedVectors
model2 = KeyedVectors.load_word2vec_format('../Words_Embedding/dsi_model_format_v4.bin', binary=True)

In [31]:
if 'science' in model2.vocab:
    print 'found it'

In [32]:
if 'science' in model.wv.vocab:
    print 'found it'

found it


In [27]:
model3 = word2vec.Word2Vec.load('dsi_model_v2.bin')

In [19]:
res = model.most_similar('analyst',topn = 100)

In [20]:
for t in res:
    print t[0], t[1]

consultant 0.561897516251
analysts 0.547333776951
scientist 0.530964612961
senior 0.503260552883
specialist 0.462088912725
manager 0.459483802319
sr 0.455553233624
consultantmanager 0.454231858253
consulting 0.448589742184
engineer 0.447730690241
analytics 0.440532177687
expert 0.434696137905
architect 0.434682279825
modelersenior 0.42735761404
director 0.417966097593
forrester 0.416917920113
statistician 0.416605740786
strategist 0.414440482855
gartner 0.409624248743
actuary 0.403312206268
researcher 0.397170811892
developer 0.397039532661
technologist 0.394302219152
modeler 0.387651234865
marketer 0.387158095837
vp 0.386842906475
advisor 0.386717766523
administrativeassistant 0.386606991291
svp 0.376260757446
pricewaterhousecoopers 0.374213129282
avp 0.374102324247
professionalsfrom 0.373046368361
errorhits 0.370910733938
intelligencestudents 0.363914638758
executives 0.361212491989
programmer 0.358302205801
deloitte 0.357511281967
sogetis 0.356489658356
officer 0.355938315392
rjmetr